In [ ]:
# This is a Python

%%bash
apt-get update && apt-get install ffmpeg libsm6 libxext6  -y

In [ ]:
%%bash 
apt-get install ghostscript -y

In [ ]:
import sys
!{sys.executable} -m pip install pymongo azure-storage-blob boto3 fuzzywuzzy python-Levenshtein geopy tabula-py camelot-py[cv] 'PyPDF2<3.0'

In [ ]:
from bs4 import BeautifulSoup
import numpy as np
import requests
from azure.storage.blob import BlobServiceClient, BlobClient
from pymongo import MongoClient
import json
import matplotlib 
import datetime
import time
import boto3
from io import BytesIO
from fuzzywuzzy import fuzz
from geopy.geocoders import Nominatim
import tabula
import pandas as pd
import PyPDF2
print("PyPDF2==" + PyPDF2.__version__)
import camelot

In [ ]:
# Getting the number of pages 
pdf_file = open('tsa.pdf', 'rb')
# Create a pdf reader object
pdf_reader = PyPDF2.PdfFileReader(pdf_file)
# Get the number of pages
number_of_pages = pdf_reader.numPages
# Close the pdf file
pdf_file.close()


# Extract the Data
# Camelot will extract the data in the table and will arrange it in a way that will allow the developer to backfill the data
tsa_df = pd.DataFrame()
for i in range(1, number_of_pages + 1):
    tables = camelot.read_pdf('tsa.pdf', flavor='lattice', pages=f'{i}', shift_text=['r', 'b'])
    tsa_df = tsa_df.append(tables[0].df)




In [ ]:
# Data Cleaning 
tsa_df.to_csv("tsa.csv")
tsa_df = pd.read_csv("tsa.csv")

#Assign Columns
tsa_df.columns = ["num","date","hour","airportcode","airportname","city","state","checkpoint","total"]
tsa_df.set_index('num', inplace=True)
# tsa_df.drop(tsa_df.index[range(0, 7)], axis=0, inplace=True)

# Remove all rows that contains Total
tsa_df = tsa_df[~tsa_df['total'].str.contains('Total', na=False)]
tsa_df

# Backfilling all data 
tsa_df['airportcode'].fillna(method='bfill', inplace=True)
tsa_df['airportname'].fillna(method='bfill', inplace=True)
tsa_df['city'].fillna(method='bfill', inplace=True)
tsa_df['state'].fillna(method='bfill', inplace=True)
# tsa_df['checkpoint'].fillna('Main Checkpoint', inplace=True)
tsa_df['total'].fillna(0, inplace=True)


# reset index and get first index with no nan
stagedf.reset_index(drop=True, inplace=True)
idx_date = stagedf['date'].first_valid_index()
idx_hour = stagedf['hour'].first_valid_index()

# Perform backfill only on the specified range for the date and the hour
stagedf.loc[0:idx_date, 'date'] = stagedf.loc[0:idx_date, 'date'].bfill()
stagedf.loc[0:idx_hour, 'hour'] = stagedf.loc[0:idx_hour, 'hour'].bfill()

stagedf["date"] = stagedf["date"].astype(str)

print(tsa_df.shape)

In [ ]:
def convert_time_to_int(time_str):
  """Converts a HH:mm time string to a single integer.

  Args:
    time_str: The HH:mm time string to convert.

  Returns:
    The converted time as a single integer.
  """

  hour, minute = time_str.split(":")
  hour = int(hour)
  minute = int(minute)
  return hour * 60 + minute

def convert_hour_to_int(time_str):
  """Converts a HH:mm time string to a single integer.

  Args:
    time_str: The HH:mm time string to convert.

  Returns:
    The converted time as a single integer.
  """

  hour, minute = time_str.split(":")
  hour = int(hour)
  return hour


def increment_date(date_string):
  """Increments a date string to the next day.

  Args:
    date_string: The date string to increment.

  Returns:
    The next day as a date string.
  """

  date = datetime.datetime.strptime(date_string, "%m/%d/%Y")
  next_day = date + datetime.timedelta(days=1)
  return next_day.strftime("%m/%d/%Y")


def convert_mm_yy_dd_to_yyyy_mm_dd(date_str):
  """Converts a mm/yy/dd date string to a yyyy-mm-dd date string.

  Args:
    date_str: The mm/yy/dd date string to convert.

  Returns:
    The converted date string in the format yyyy-mm-dd.
  """

  if(date_str is None):
      return date_str
  else: 
    month, year, day = date_str.split("/")
    year = str(int(year) + 2000)
    return f"{year}-{month}-{day}"

def increment_time(time_str):
  """Increments a 00:00 time string to 01:00.

  Args:
    time_str: The 00:00 time string to increment.

  Returns:
    The incremented time string in the format hh:mm.
  """

  hour, minute = time_str.split(":")
  hour = int(hour) + 1
  minute = "00" if minute == "00" else minute
  return f"{hour}:{minute}"


#def main():
#  df = pd.DataFrame({
#    "time": ["00:00", "00:01", "00:02"]
#  })

#  df["time_incremented"] = df["time"].apply(increment_time)

#  print(df)

def replace_nan_with_value(df, column_name,custom_value):
  """Replaces all NaN values in a DataFrame column with 00/00/0000.

  Args:
    df: The DataFrame to replace the NaN values in.
    column_name: The name of the column to replace the NaN values in.

  Returns:
    The DataFrame with the NaN values replaced.
  """

  df[column_name].replace(to_replace=np.nan, value=custom_value, inplace=True)
  return df

In [ ]:
# Algorithm that will rearrange the table

previous_code = None
for index, row in stagedf.iterrows():
    print(index)
    current_date = row['date']
    current_hour = row['hour']
    airport_code = row['airportcode']
    print(current_date)
 #   if(index == 844):
 #       break
    if(airport_code.startswith('A') and current_date =="nan" and (previous_code is None or not previous_code.startswith('A'))):
        first_a = True
        print("Airport Starts with A")
        print(index)
        if(stagedf.at[index-1, 'hour'] == "23:00"):
            print("Hour is 23")
            print(index)
            stagedf.at[index, 'hour'] = "00:00"
            stagedf.at[index, 'date'] = increment_date(stagedf.at[index-1, 'date'])
        else:
            print("Hour is not 23")
            print(index)
            stagedf.at[index, 'date'] = stagedf.at[index-1, 'date']
            stagedf.at[index, 'hour'] = increment_time(stagedf.at[index-1, 'hour'])
    else:
        print("Airport does not starts with A")
        if(index < 30):
            print("Index less than 30")
            continue
        stagedf.at[index, 'date'] = stagedf.at[index-1, 'date']
        stagedf.at[index, 'hour'] = stagedf.at[index-1, 'hour']
    previous_code = airport_code

In [ ]:
stagedf.to_csv("final.pdf")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ca436d51-26cb-4a7b-9af3-0ce82bc99b1b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>